In [1]:
!pip install -U langchain-openai openai --quiet
!pip install -U langchain-openai openai

In [2]:
import os
import logging
import time
import re
import json
from typing import Dict, List, Any, Optional, Tuple
from datetime import datetime
from dataclasses import dataclass
from enum import Enum
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage



In [3]:
"""
Diagnostic Italian VAT Compliance Assessment Tool
Fixed version with improved scoring and detailed diagnostics


"""


# Configure diagnostic logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class QualityLevel(Enum):
    """Quality classification levels"""
    EXCELLENT = "Excellent"
    GOOD = "Good"
    SATISFACTORY = "Satisfactory"
    NEEDS_IMPROVEMENT = "Needs Improvement"
    INADEQUATE = "Inadequate"

@dataclass
class EnhancedAssessmentCriteria:
    """Enhanced assessment criteria with semantic flexibility"""

    def __post_init__(self):
        # Core compliance terms (more flexible)
        self.compliance_terms = [
            "obbligo", "obbligh", "adempiment", "dovere", "necessit",
            "sanzione", "sanzion", "penali", "multa", "ammenda",
            "termine", "scadenz", "deadline", "entro", "tempo",
            "violazione", "violazion", "infrazione", "trasgressione",
            "ritardo", "ritard", "inosservanz", "inadempiment"
        ]

        # Technical VAT terms (with variations)
        self.technical_terms = [
            "fatturazione elettronica", "fatturazione digitale", "e-fattur", "fattur",
            "liquidazione iva", "liquidazion", "versamento iva",
            "registro iva", "registr", "registrazione",
            "sistema di interscambio", "sdi", "interscambio",
            "agenzia delle entrate", "agenzia entrate", "fisco",
            "split payment", "reverse charge", "autofattur",
            "dichiarazione iva", "dichiarazion", "comunicazione periodica"
        ]

        # Legal references (flexible matching)
        self.legal_refs = [
            "dpr 633", "dpr633", "decreto 633", "codice iva",
            "art.", "articolo", "comma", "decreto", "legge",
            "normativa", "regolament", "disposizion"
        ]

        # Precision indicators
        self.precision_terms = [
            "euro", "€", "percent", "%", "giorni", "mese", "mesi",
            "entro", "termine", "scadenz", "minimo", "massimo",
            "dal", "fino", "tra", "mensile", "trimestrale"
        ]

        # Accuracy indicators
        self.accuracy_terms = [
            "2024", "srl", "società", "responsabilità limitata",
            "regime ordinario", "standard", "italia", "italiana", "italiano"
        ]

class DiagnosticVATAssessor:
    """Enhanced assessor with detailed diagnostics"""

    def __init__(self, api_key: Optional[str] = None):
        self.api_key = api_key or self._get_api_key()
        self.criteria = EnhancedAssessmentCriteria()

        # Rebalanced weights (less strict)
        self.assessment_weights = {
            "compliance_terms": 0.20,      # Reduced from 0.25
            "technical_terms": 0.25,       # Keep high for expertise
            "legal_refs": 0.15,           # Reduced from 0.20
            "precision_terms": 0.25,       # Increased for completeness
            "accuracy_terms": 0.15        # Increased from 0.10
        }

        logger.info("Diagnostic VAT Assessor initialized with enhanced criteria")

    def _get_api_key(self) -> str:
        """Securely obtain API key"""
        if "OPENAI_API_KEY" in os.environ:
            return os.environ["OPENAI_API_KEY"]

        import getpass
        return getpass.getpass("Enter OpenAI API Key: ")

    def get_enhanced_prompt(self) -> str:
        """Enhanced prompt for better Italian responses"""
        return """
Sei un consulente fiscale senior specializzato nella compliance IVA italiana per le SRL.

COMPITO: Fornisci una panoramica completa dei 5 adempimenti IVA più critici per una SRL italiana in regime ordinario nel 2024.

REQUISITI:
1. Elenca 5 obblighi IVA principali in ordine di importanza
2. Per ogni obbligo, includi:
   - Il requisito legale specifico
   - Scadenze e frequenze applicabili
   - Conseguenze della non conformità (sanzioni, penalità)
3. Usa terminologia legale e fiscale italiana appropriata
4. Includi riferimenti normativi pertinenti
5. Struttura la risposta come punti numerati con linguaggio chiaro e professionale

IMPORTANTE: Rispondi SOLO in italiano usando linguaggio formale appropriato per documentazione fiscale aziendale.

Fornisci informazioni precise e aggiornate al 2024 per SRL in regime ordinario IVA.
"""

    def get_enhanced_system_prompt(self) -> str:
        """Enhanced system prompt"""
        return """
Sei un esperto consulente fiscale italiano con competenze approfondite in:
- Legislazione IVA italiana (DPR 633/72 e successive modifiche)
- Compliance fiscale per SRL italiane
- Normative IVA 2024 correnti e requisiti
- Procedure dell'Agenzia delle Entrate
- Fatturazione elettronica e Sistema di Interscambio (SdI)

Fornisci consigli accurati e professionali usando terminologia legale italiana appropriata e normative correnti.
Il tuo obiettivo è fornire informazioni complete, precise e praticamente utilizzabili per la compliance aziendale.
"""

    def flexible_term_search(self, content: str, terms: List[str]) -> Tuple[int, List[str]]:
        """Enhanced search with partial matching and stemming"""
        content_lower = content.lower()
        found_terms = []

        for term in terms:
            term_lower = term.lower()

            # Exact match
            if term_lower in content_lower:
                found_terms.append(term)
                continue

            # Partial match for compound terms
            if len(term_lower.split()) > 1:
                words = term_lower.split()
                if all(word in content_lower for word in words):
                    found_terms.append(term)
                    continue

            # Stem matching (simple)
            if len(term_lower) > 5:
                stem = term_lower[:5]
                if stem in content_lower:
                    found_terms.append(term)

        return len(set(found_terms)), list(set(found_terms))

    def assess_response_enhanced(self, response_content: str) -> Dict[str, Any]:
        """Enhanced assessment with detailed diagnostics"""

        # Get criteria as lists
        criteria_dict = {
            "compliance_terms": self.criteria.compliance_terms,
            "technical_terms": self.criteria.technical_terms,
            "legal_refs": self.criteria.legal_refs,
            "precision_terms": self.criteria.precision_terms,
            "accuracy_terms": self.criteria.accuracy_terms
        }

        category_results = {}
        total_weighted_score = 0
        diagnostic_info = {
            "content_preview": response_content[:200] + "..." if len(response_content) > 200 else response_content,
            "content_length": len(response_content),
            "language_detected": self._detect_language(response_content),
            "found_terms_by_category": {}
        }

        for category, terms in criteria_dict.items():
            matches, found_terms = self.flexible_term_search(response_content, terms)
            coverage_rate = matches / len(terms) if terms else 0
            category_score = coverage_rate * 100
            weighted_contribution = category_score * self.assessment_weights[category]

            category_results[category] = {
                "matches": matches,
                "total_terms": len(terms),
                "coverage_rate": round(coverage_rate, 3),
                "category_score": round(category_score, 1),
                "weight": self.assessment_weights[category],
                "weighted_contribution": round(weighted_contribution, 2),
                "found_terms": found_terms[:5]  # Show first 5 found terms
            }

            diagnostic_info["found_terms_by_category"][category] = found_terms
            total_weighted_score += weighted_contribution

        # Enhanced structural analysis
        structure_analysis = self._analyze_structure_enhanced(response_content)

        # Quality bonuses (more generous)
        bonus_score = 0
        bonus_details = []

        # Structure bonus
        if structure_analysis["numbered_items"] >= 5:
            bonus_score += 10
            bonus_details.append("Complete 5-point structure (+10%)")
        elif structure_analysis["numbered_items"] >= 3:
            bonus_score += 5
            bonus_details.append(f"Good structure {structure_analysis['numbered_items']} points (+5%)")

        # Content depth bonus
        if structure_analysis["word_count"] >= 200:
            bonus_score += 8
            bonus_details.append("Comprehensive content (+8%)")
        elif structure_analysis["word_count"] >= 100:
            bonus_score += 4
            bonus_details.append("Adequate detail (+4%)")

        # Italian language bonus
        if diagnostic_info["language_detected"] == "italian":
            bonus_score += 12
            bonus_details.append("Proper Italian language (+12%)")

        # Legal precision bonus
        legal_indicators = len(re.findall(r'art\.|articolo|comma|dpr|decreto', response_content.lower()))
        if legal_indicators >= 3:
            bonus_score += 8
            bonus_details.append(f"Strong legal references ({legal_indicators}) (+8%)")
        elif legal_indicators >= 1:
            bonus_score += 4
            bonus_details.append(f"Legal references present ({legal_indicators}) (+4%)")

        # Monetary precision bonus
        monetary_indicators = len(re.findall(r'€|euro|\d+%|percent|minimo|massimo', response_content.lower()))
        if monetary_indicators >= 3:
            bonus_score += 6
            bonus_details.append(f"Good monetary precision ({monetary_indicators}) (+6%)")

        # Final score calculation
        final_score = min(100, max(0, total_weighted_score + bonus_score))
        quality_level = self._determine_quality_level(final_score)

        return {
            "overall_score": round(final_score, 1),
            "base_score": round(total_weighted_score, 1),
            "bonus_score": round(bonus_score, 1),
            "quality_level": quality_level.value,
            "category_results": category_results,
            "structure_analysis": structure_analysis,
            "bonus_details": bonus_details,
            "diagnostic_info": diagnostic_info,
            "assessment_timestamp": datetime.now().isoformat()
        }

    def _detect_language(self, content: str) -> str:
        """Simple language detection"""
        italian_indicators = ["della", "degli", "con", "per", "che", "una", "del", "nel", "sul"]
        english_indicators = ["the", "and", "for", "with", "that", "this", "from", "they"]

        content_lower = content.lower()
        italian_count = sum(1 for word in italian_indicators if word in content_lower)
        english_count = sum(1 for word in english_indicators if word in content_lower)

        if italian_count > english_count:
            return "italian"
        elif english_count > italian_count:
            return "english"
        else:
            return "mixed"

    def _analyze_structure_enhanced(self, content: str) -> Dict[str, Any]:
        """Enhanced structural analysis"""
        lines = [line.strip() for line in content.split('\n') if line.strip()]
        numbered_items = len(re.findall(r'^\d+\.', content, re.MULTILINE))
        sentence_count = len(re.findall(r'[.!?]+', content))
        word_count = len(content.split())

        # Additional metrics
        paragraph_count = len([line for line in lines if len(line) > 50])
        bullet_points = len(re.findall(r'^[-•*]\s', content, re.MULTILINE))

        return {
            "total_lines": len(lines),
            "numbered_items": numbered_items,
            "bullet_points": bullet_points,
            "paragraph_count": paragraph_count,
            "sentence_count": sentence_count,
            "word_count": word_count,
            "avg_sentence_length": round(word_count / max(sentence_count, 1), 1),
            "has_proper_structure": numbered_items >= 3 or bullet_points >= 3
        }

    def _determine_quality_level(self, score: float) -> QualityLevel:
        """Determine quality level with adjusted thresholds"""
        if score >= 85:
            return QualityLevel.EXCELLENT
        elif score >= 70:
            return QualityLevel.GOOD
        elif score >= 55:
            return QualityLevel.SATISFACTORY
        elif score >= 40:
            return QualityLevel.NEEDS_IMPROVEMENT
        else:
            return QualityLevel.INADEQUATE

    def generate_response(
        self,
        model: str = "gpt-4",
        temperature: float = 0.2,  # Lower for more consistent Italian
        max_tokens: int = 1200,
        timeout: int = 60
    ) -> Dict[str, Any]:
        """Generate response with enhanced parameters"""
        try:
            llm = ChatOpenAI(
                model_name=model,
                temperature=temperature,
                max_tokens=max_tokens,
                timeout=timeout,
                openai_api_key=self.api_key
            )

            messages = [
                SystemMessage(content=self.get_enhanced_system_prompt()),
                HumanMessage(content=self.get_enhanced_prompt())
            ]

            start_time = time.time()
            response = llm.invoke(messages)
            response_time = time.time() - start_time

            result = {
                "content": response.content,
                "model": model,
                "temperature": temperature,
                "response_time": round(response_time, 2),
                "timestamp": datetime.now().isoformat(),
                "token_count": len(response.content.split())
            }

            logger.info(f"Response generated successfully with {model}")
            return result

        except Exception as e:
            logger.error(f"Error generating response: {str(e)}")
            raise

    def run_diagnostic_assessment(
        self,
        models: List[str] = None,
        iterations: int = 3,
        verbose: bool = True
    ) -> Dict[str, Any]:
        """Run assessment with detailed diagnostics"""

        if models is None:
            models = ["gpt-3.5-turbo", "gpt-4"]

        if verbose:
            print("🔬 Diagnostic Italian VAT Compliance Assessment")
            print("=" * 60)
            print(f"Models: {', '.join(models)}")
            print(f"Iterations per model: {iterations}")
            print("=" * 60)

        results = {}

        for model in models:
            if verbose:
                print(f"\n🧪 Testing {model}...")

            model_results = []

            for iteration in range(iterations):
                if verbose:
                    print(f"  Iteration {iteration + 1}/{iterations}...", end=" ")

                try:
                    # Generate response
                    response = self.generate_response(model=model)

                    # Enhanced assessment
                    assessment = self.assess_response_enhanced(response["content"])

                    # Combine results
                    combined_result = {**response, **assessment}
                    model_results.append(combined_result)

                    if verbose:
                        print(f"Score: {assessment['overall_score']}% ({assessment['quality_level']})")
                        print(f"    Language: {assessment['diagnostic_info']['language_detected']}")
                        print(f"    Structure: {assessment['structure_analysis']['numbered_items']} points")
                        print(f"    Bonuses: {', '.join(assessment['bonus_details']) if assessment['bonus_details'] else 'None'}")

                except Exception as e:
                    if verbose:
                        print(f"FAILED: {str(e)}")
                    model_results.append({
                        "error": str(e),
                        "model": model,
                        "iteration": iteration + 1
                    })

            # Calculate statistics
            successful_results = [r for r in model_results if "error" not in r]

            if successful_results:
                scores = [r["overall_score"] for r in successful_results]
                results[model] = {
                    "individual_results": model_results,
                    "statistics": {
                        "mean_score": round(sum(scores) / len(scores), 1),
                        "min_score": min(scores),
                        "max_score": max(scores),
                        "std_dev": round(self._calculate_std_dev(scores), 2),
                        "success_rate": len(successful_results) / iterations
                    },
                    "diagnostic_summary": self._create_diagnostic_summary(successful_results)
                }
            else:
                results[model] = {
                    "individual_results": model_results,
                    "statistics": {"error": "All iterations failed"}
                }

        return {
            "assessment_summary": results,
            "metadata": {
                "assessment_date": datetime.now().isoformat(),
                "models_tested": models,
                "iterations_per_model": iterations,
                "total_assessments": len(models) * iterations,
                "scoring_version": "enhanced_v1.1"
            }
        }

    def _create_diagnostic_summary(self, results: List[Dict[str, Any]]) -> Dict[str, Any]:
        """Create diagnostic summary from results"""

        # Language distribution
        languages = [r["diagnostic_info"]["language_detected"] for r in results]
        language_dist = {lang: languages.count(lang) for lang in set(languages)}

        # Average structure metrics
        avg_structure = {
            "avg_numbered_items": round(sum(r["structure_analysis"]["numbered_items"] for r in results) / len(results), 1),
            "avg_word_count": round(sum(r["structure_analysis"]["word_count"] for r in results) / len(results), 1),
            "proper_structure_rate": sum(1 for r in results if r["structure_analysis"]["has_proper_structure"]) / len(results)
        }

        # Most common bonus types
        all_bonuses = []
        for r in results:
            all_bonuses.extend(r.get("bonus_details", []))

        bonus_freq = {}
        for bonus in all_bonuses:
            bonus_type = bonus.split("(")[0].strip()
            bonus_freq[bonus_type] = bonus_freq.get(bonus_type, 0) + 1

        return {
            "language_distribution": language_dist,
            "average_structure": avg_structure,
            "common_bonuses": bonus_freq
        }

    def _calculate_std_dev(self, values: List[float]) -> float:
        """Calculate standard deviation"""
        if len(values) < 2:
            return 0.0

        mean = sum(values) / len(values)
        variance = sum((x - mean) ** 2 for x in values) / (len(values) - 1)
        return variance ** 0.5

    def display_enhanced_report(self, results: Dict[str, Any]) -> None:
        """Display enhanced diagnostic report"""

        print(f"\n{'='*70}")
        print("🔬 ENHANCED DIAGNOSTIC REPORT")
        print(f"{'='*70}")

        metadata = results["metadata"]
        print(f"Assessment Date: {metadata['assessment_date']}")
        print(f"Models Tested: {', '.join(metadata['models_tested'])}")
        print(f"Scoring Version: {metadata['scoring_version']}")

        print(f"\n📊 MODEL COMPARISON:")

        for model, data in results["assessment_summary"].items():
            if "statistics" in data and "error" not in data["statistics"]:
                stats = data["statistics"]
                diag = data["diagnostic_summary"]

                print(f"\n🤖 {model.upper()}:")
                print(f"   Mean Score: {stats['mean_score']}% (σ: {stats['std_dev']})")
                print(f"   Score Range: {stats['min_score']}% - {stats['max_score']}%")
                print(f"   Success Rate: {stats['success_rate'] * 100:.0f}%")

                print(f"   📝 Content Quality:")
                print(f"     Avg Words: {diag['average_structure']['avg_word_count']}")
                print(f"     Avg Structure Points: {diag['average_structure']['avg_numbered_items']}")
                print(f"     Proper Structure Rate: {diag['average_structure']['proper_structure_rate'] * 100:.0f}%")

                print(f"   🌍 Language Distribution:")
                for lang, count in diag['language_distribution'].items():
                    print(f"     {lang.title()}: {count}/{len(data['individual_results'])}")

                if diag['common_bonuses']:
                    print(f"   🏆 Common Bonuses:")
                    for bonus, freq in diag['common_bonuses'].items():
                        print(f"     {bonus}: {freq} times")

        print(f"\n💡 DIAGNOSTIC INSIGHTS:")

        # Compare models
        model_scores = {}
        for model, data in results["assessment_summary"].items():
            if "statistics" in data and "error" not in data["statistics"]:
                model_scores[model] = data["statistics"]["mean_score"]

        if len(model_scores) >= 2:
            best_model = max(model_scores.keys(), key=lambda x: model_scores[x])
            worst_model = min(model_scores.keys(), key=lambda x: model_scores[x])

            print(f"   🥇 Best Performer: {best_model} ({model_scores[best_model]}%)")
            print(f"   📈 Performance Gap: {model_scores[best_model] - model_scores[worst_model]:.1f}%")

        # Check for issues
        all_results = []
        for model_data in results["assessment_summary"].values():
            if "individual_results" in model_data:
                all_results.extend([r for r in model_data["individual_results"] if "error" not in r])

        if all_results:
            avg_score = sum(r["overall_score"] for r in all_results) / len(all_results)
            italian_responses = sum(1 for r in all_results if r["diagnostic_info"]["language_detected"] == "italian")

            print(f"   📊 Overall Average: {avg_score:.1f}%")
            print(f"   🇮🇹 Italian Response Rate: {italian_responses}/{len(all_results)} ({italian_responses/len(all_results)*100:.0f}%)")

            if avg_score < 50:
                print(f"   ⚠️  Low scores detected - consider prompt optimization")
            if italian_responses < len(all_results) * 0.8:
                print(f"   ⚠️  Language compliance issue - enhance Italian prompting")

def main():
    """Enhanced main function with diagnostics"""

    print("🔬 Diagnostic Italian VAT Compliance Assessment Tool")
    print("Enhanced version with detailed scoring diagnostics")
    print("=" * 65)

    # Initialize enhanced assessor
    assessor = DiagnosticVATAssessor()

    # Run diagnostic assessment
    results = assessor.run_diagnostic_assessment(
        models=["gpt-3.5-turbo", "gpt-4"],
        iterations=3,
        verbose=True
    )

    # Display enhanced report
    assessor.display_enhanced_report(results)

    # Save results with diagnostics
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"diagnostic_vat_assessment_{timestamp}.json"

    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=2, ensure_ascii=False)

    print(f"\n💾 Detailed diagnostic results saved to: {filename}")

    return results

if __name__ == "__main__":
    main()

🔬 Diagnostic Italian VAT Compliance Assessment Tool
Enhanced version with detailed scoring diagnostics
Enter OpenAI API Key: ··········
🔬 Diagnostic Italian VAT Compliance Assessment
Models: gpt-3.5-turbo, gpt-4
Iterations per model: 3

🧪 Testing gpt-3.5-turbo...
  Iteration 1/3... Score: 86.4% (Excellent)
    Language: italian
    Structure: 5 points
    Bonuses: Complete 5-point structure (+10%), Comprehensive content (+8%), Proper Italian language (+12%), Legal references present (1) (+4%)
  Iteration 2/3... Score: 88.5% (Excellent)
    Language: italian
    Structure: 5 points
    Bonuses: Complete 5-point structure (+10%), Comprehensive content (+8%), Proper Italian language (+12%), Legal references present (2) (+4%)
  Iteration 3/3... Score: 94.5% (Excellent)
    Language: italian
    Structure: 5 points
    Bonuses: Complete 5-point structure (+10%), Comprehensive content (+8%), Proper Italian language (+12%), Strong legal references (4) (+8%)

🧪 Testing gpt-4...
  Iteration 1/3

In [4]:
class VATResultsViewer:
    """Professional viewer for VAT assessment results with sample display"""

    def __init__(self, results_file: Optional[str] = None):
        """
        Initialize viewer with optional results file

        Args:
            results_file: Path to JSON results file. If None, will look for latest.
        """
        self.results_file = results_file or self._find_latest_results()
        self.results = self._load_results()

    def _find_latest_results(self) -> str:
        """Find the most recent diagnostic results file"""
        json_files = [f for f in os.listdir('.') if f.startswith('diagnostic_vat_assessment_') and f.endswith('.json')]
        if not json_files:
            raise FileNotFoundError("No diagnostic results files found. Run assessment first.")

        # Sort by timestamp in filename
        json_files.sort(reverse=True)
        return json_files[0]

    def _load_results(self) -> Dict[str, Any]:
        """Load results from JSON file"""
        try:
            with open(self.results_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        except Exception as e:
            raise ValueError(f"Error loading results file {self.results_file}: {e}")

    def display_sample_responses(self, model: str = None, max_samples: int = 3) -> None:
        """
        Display sample responses from specified model or all models

        Args:
            model: Specific model to show (e.g., 'gpt-4'). If None, shows all.
            max_samples: Maximum number of samples to display per model
        """
        print("🔍 SAMPLE GENERATED RESPONSES")
        print("=" * 70)

        assessment_data = self.results.get("assessment_summary", {})

        models_to_show = [model] if model else list(assessment_data.keys())

        for model_name in models_to_show:
            if model_name not in assessment_data:
                print(f"❌ Model '{model_name}' not found in results")
                continue

            model_data = assessment_data[model_name]
            individual_results = model_data.get("individual_results", [])

            # Filter successful results
            successful_results = [r for r in individual_results if "error" not in r]

            if not successful_results:
                print(f"❌ No successful results for {model_name}")
                continue

            print(f"\n🤖 {model_name.upper()} - SAMPLE RESPONSES")
            print("-" * 50)

            # Show up to max_samples
            samples_to_show = min(max_samples, len(successful_results))

            for i, result in enumerate(successful_results[:samples_to_show]):
                self._display_single_response(result, i + 1, model_name)

    def _display_single_response(self, result: Dict[str, Any], sample_num: int, model: str) -> None:
        """Display a single response with detailed analysis"""

        print(f"\n📄 SAMPLE {sample_num} - {model}")
        print("─" * 40)

        # Basic metrics
        print(f"Score: {result.get('overall_score', 'N/A')}% ({result.get('quality_level', 'N/A')})")
        print(f"Response Time: {result.get('response_time', 'N/A')}s")
        print(f"Word Count: {result.get('structure_analysis', {}).get('word_count', 'N/A')}")
        print(f"Language: {result.get('diagnostic_info', {}).get('language_detected', 'N/A').title()}")

        # Bonuses achieved
        bonuses = result.get('bonus_details', [])
        if bonuses:
            print(f"Bonuses: {', '.join(bonuses)}")

        print(f"\n📝 GENERATED CONTENT:")
        print("─" * 30)
        content = result.get('content', 'No content available')

        # Format content nicely
        formatted_content = self._format_content(content)
        print(formatted_content)

        # Category breakdown
        print(f"\n📊 CATEGORY PERFORMANCE:")
        category_results = result.get('category_results', {})
        for category, data in category_results.items():
            category_name = category.replace('_', ' ').title()
            score = data.get('category_score', 0)
            matches = data.get('matches', 0)
            total = data.get('total_terms', 0)
            print(f"  • {category_name}: {score}% ({matches}/{total} terms)")

        print("─" * 40)

    def _format_content(self, content: str) -> str:
        """Format content for better readability"""
        # Add some basic formatting
        lines = content.split('\n')
        formatted_lines = []

        for line in lines:
            line = line.strip()
            if not line:
                continue

            # Highlight numbered points
            if re.match(r'^\d+\.', line):
                formatted_lines.append(f"🔹 {line}")
            else:
                formatted_lines.append(f"   {line}")

        return '\n'.join(formatted_lines)

    def compare_responses(self, models: List[str] = None) -> None:
        """
        Compare responses side by side between models

        Args:
            models: List of models to compare. If None, compares all available.
        """
        assessment_data = self.results.get("assessment_summary", {})

        if models is None:
            models = list(assessment_data.keys())

        print("🔄 RESPONSE COMPARISON")
        print("=" * 70)

        # Get best response from each model
        model_best = {}
        for model in models:
            if model in assessment_data:
                successful_results = [r for r in assessment_data[model].get("individual_results", [])
                                    if "error" not in r]
                if successful_results:
                    # Get highest scoring response
                    best_response = max(successful_results, key=lambda x: x.get('overall_score', 0))
                    model_best[model] = best_response

        if len(model_best) < 2:
            print("❌ Need at least 2 models with successful results for comparison")
            return

        # Display comparison
        for model, response in model_best.items():
            print(f"\n🏆 BEST FROM {model.upper()}")
            print(f"Score: {response.get('overall_score')}% | Words: {response.get('structure_analysis', {}).get('word_count', 'N/A')}")
            print("─" * 50)

            # Show first 300 characters of content
            content = response.get('content', '')
            preview = content[:300] + "..." if len(content) > 300 else content
            print(self._format_content(preview))
            print()

    def analyze_patterns(self) -> None:
        """Analyze patterns across all responses"""

        print("🧠 PATTERN ANALYSIS")
        print("=" * 50)

        all_responses = []
        assessment_data = self.results.get("assessment_summary", {})

        # Collect all successful responses
        for model_data in assessment_data.values():
            successful_results = [r for r in model_data.get("individual_results", [])
                                if "error" not in r]
            all_responses.extend(successful_results)

        if not all_responses:
            print("❌ No successful responses to analyze")
            return

        # Analyze score distribution
        scores = [r.get('overall_score', 0) for r in all_responses]
        print(f"📊 Score Distribution:")
        print(f"   Average: {sum(scores)/len(scores):.1f}%")
        print(f"   Range: {min(scores):.1f}% - {max(scores):.1f}%")
        print(f"   Excellent (≥85%): {sum(1 for s in scores if s >= 85)}/{len(scores)} ({sum(1 for s in scores if s >= 85)/len(scores)*100:.0f}%)")

        # Analyze common terms found
        print(f"\n🔍 Common Successfully Detected Terms:")
        term_frequency = {}

        for response in all_responses:
            diagnostic_info = response.get('diagnostic_info', {})
            found_terms = diagnostic_info.get('found_terms_by_category', {})

            for category_terms in found_terms.values():
                for term in category_terms:
                    term_frequency[term] = term_frequency.get(term, 0) + 1

        # Show top 10 most frequent terms
        top_terms = sorted(term_frequency.items(), key=lambda x: x[1], reverse=True)[:10]
        for term, freq in top_terms:
            print(f"   • {term}: {freq}/{len(all_responses)} responses ({freq/len(all_responses)*100:.0f}%)")

        # Analyze structure patterns
        print(f"\n📋 Structure Patterns:")
        numbered_items = [r.get('structure_analysis', {}).get('numbered_items', 0) for r in all_responses]
        word_counts = [r.get('structure_analysis', {}).get('word_count', 0) for r in all_responses]

        print(f"   Average numbered items: {sum(numbered_items)/len(numbered_items):.1f}")
        print(f"   Average word count: {sum(word_counts)/len(word_counts):.0f}")
        print(f"   Perfect structure (5 points): {sum(1 for n in numbered_items if n == 5)}/{len(numbered_items)} ({sum(1 for n in numbered_items if n == 5)/len(numbered_items)*100:.0f}%)")

    def export_samples(self, output_file: str = None, format: str = "markdown") -> str:
        """
        Export sample responses to file

        Args:
            output_file: Output filename. If None, generates timestamp-based name.
            format: Export format ('markdown', 'txt', 'html')

        Returns:
            Path to exported file
        """
        if output_file is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            extension = "md" if format == "markdown" else format
            output_file = f"vat_response_samples_{timestamp}.{extension}"

        assessment_data = self.results.get("assessment_summary", {})

        if format == "markdown":
            content = self._generate_markdown_export(assessment_data)
        elif format == "txt":
            content = self._generate_text_export(assessment_data)
        elif format == "html":
            content = self._generate_html_export(assessment_data)
        else:
            raise ValueError(f"Unsupported format: {format}")

        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(content)

        print(f"📁 Samples exported to: {output_file}")
        return output_file

    def _generate_markdown_export(self, assessment_data: Dict[str, Any]) -> str:
        """Generate markdown export of samples"""

        content = ["# Italian VAT Compliance - Generated Response Samples\n"]
        content.append(f"**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        content.append(f"**Source:** {self.results_file}\n\n")

        for model, data in assessment_data.items():
            successful_results = [r for r in data.get("individual_results", []) if "error" not in r]

            if not successful_results:
                continue

            content.append(f"## {model.upper()} Samples\n")

            # Show best response
            best_response = max(successful_results, key=lambda x: x.get('overall_score', 0))

            content.append(f"**Best Performance:** {best_response.get('overall_score')}% ({best_response.get('quality_level')})\n")
            content.append(f"**Response Time:** {best_response.get('response_time')}s\n")
            content.append(f"**Word Count:** {best_response.get('structure_analysis', {}).get('word_count')}\n\n")

            content.append("### Generated Content:\n\n")
            content.append("```\n")
            content.append(best_response.get('content', 'No content'))
            content.append("\n```\n\n")

            # Category breakdown
            content.append("### Category Performance:\n\n")
            category_results = best_response.get('category_results', {})
            for category, cat_data in category_results.items():
                category_name = category.replace('_', ' ').title()
                score = cat_data.get('category_score', 0)
                matches = cat_data.get('matches', 0)
                total = cat_data.get('total_terms', 0)
                content.append(f"- **{category_name}:** {score}% ({matches}/{total} terms)\n")

            content.append("\n")

        return ''.join(content)

    def _generate_text_export(self, assessment_data: Dict[str, Any]) -> str:
        """Generate plain text export"""
        lines = [
            "ITALIAN VAT COMPLIANCE - GENERATED RESPONSE SAMPLES",
            "=" * 60,
            f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
            f"Source: {self.results_file}",
            ""
        ]

        for model, data in assessment_data.items():
            successful_results = [r for r in data.get("individual_results", []) if "error" not in r]

            if not successful_results:
                continue

            best_response = max(successful_results, key=lambda x: x.get('overall_score', 0))

            lines.extend([
                f"{model.upper()} BEST SAMPLE",
                "-" * 30,
                f"Score: {best_response.get('overall_score')}%",
                f"Quality: {best_response.get('quality_level')}",
                f"Words: {best_response.get('structure_analysis', {}).get('word_count')}",
                "",
                "CONTENT:",
                best_response.get('content', 'No content'),
                "",
                "=" * 60,
                ""
            ])

        return '\n'.join(lines)

    def _generate_html_export(self, assessment_data: Dict[str, Any]) -> str:
        """Generate HTML export"""
        html_content = f"""
<!DOCTYPE html>
<html>
<head>
    <title>VAT Compliance Response Samples</title>
    <style>
        body {{ font-family: Arial, sans-serif; margin: 40px; }}
        .sample {{ border: 1px solid #ddd; margin: 20px 0; padding: 20px; }}
        .score {{ font-size: 18px; color: #2e8b57; font-weight: bold; }}
        .content {{ background: #f5f5f5; padding: 15px; white-space: pre-wrap; }}
        .metrics {{ display: grid; grid-template-columns: 1fr 1fr; gap: 10px; }}
    </style>
</head>
<body>
    <h1>🏛️ Italian VAT Compliance - Response Samples</h1>
    <p><strong>Generated:</strong> {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>
    <p><strong>Source:</strong> {self.results_file}</p>
"""

        for model, data in assessment_data.items():
            successful_results = [r for r in data.get("individual_results", []) if "error" not in r]

            if not successful_results:
                continue

            best_response = max(successful_results, key=lambda x: x.get('overall_score', 0))

            html_content += f"""
    <div class="sample">
        <h2>🤖 {model.upper()}</h2>
        <div class="score">Score: {best_response.get('overall_score')}% ({best_response.get('quality_level')})</div>
        <div class="metrics">
            <div>Response Time: {best_response.get('response_time')}s</div>
            <div>Word Count: {best_response.get('structure_analysis', {}).get('word_count')}</div>
        </div>

        <h3>Generated Content:</h3>
        <div class="content">{best_response.get('content', 'No content')}</div>
    </div>
"""

        html_content += """
</body>
</html>
"""
        return html_content

# Convenience functions for quick usage
def view_samples(results_file: str = None, model: str = None, max_samples: int = 3):
    """Quick function to view samples"""
    viewer = VATResultsViewer(results_file)
    viewer.display_sample_responses(model, max_samples)

def compare_models(results_file: str = None, models: List[str] = None):
    """Quick function to compare models"""
    viewer = VATResultsViewer(results_file)
    viewer.compare_responses(models)

def analyze_all(results_file: str = None):
    """Quick function for full analysis"""
    viewer = VATResultsViewer(results_file)
    print("📋 Full Analysis Report")
    print("=" * 50)
    viewer.display_sample_responses(max_samples=2)
    viewer.compare_responses()
    viewer.analyze_patterns()

def export_samples(results_file: str = None, format: str = "markdown"):
    """Quick function to export samples"""
    viewer = VATResultsViewer(results_file)
    return viewer.export_samples(format=format)

# Main execution for testing
if __name__ == "__main__":
    print("🔍 VAT Results Viewer")
    print("Available functions:")
    print("- view_samples(): Display sample responses")
    print("- compare_models(): Compare model responses")
    print("- analyze_all(): Full analysis report")
    print("- export_samples(): Export to file")

    # Example usage
    try:
        viewer = VATResultsViewer()
        print(f"\nLoaded results from: {viewer.results_file}")

        # Show quick sample
        viewer.display_sample_responses(max_samples=1)

    except FileNotFoundError as e:
        print(f"❌ {e}")
        print("Run the diagnostic assessment first to generate results.")

🔍 VAT Results Viewer
Available functions:
- view_samples(): Display sample responses
- compare_models(): Compare model responses
- analyze_all(): Full analysis report
- export_samples(): Export to file

Loaded results from: diagnostic_vat_assessment_20250625_095631.json
🔍 SAMPLE GENERATED RESPONSES

🤖 GPT-3.5-TURBO - SAMPLE RESPONSES
--------------------------------------------------

📄 SAMPLE 1 - gpt-3.5-turbo
────────────────────────────────────────
Score: 86.4% (Excellent)
Response Time: 8.18s
Word Count: 393
Language: Italian
Bonuses: Complete 5-point structure (+10%), Comprehensive content (+8%), Proper Italian language (+12%), Legal references present (1) (+4%)

📝 GENERATED CONTENT:
──────────────────────────────
   Certamente, ecco una panoramica dei 5 adempimenti IVA più critici per una SRL italiana in regime ordinario nel 2024:
🔹 1. **Registrazione IVA e Comunicazioni Obbligatorie:**
   - Requisito legale: Ogni SRL deve registrarsi presso l'Agenzia delle Entrate per l'IVA entr